In [1]:
import pandas as pd
import cohere
import time
from annoy import AnnoyIndex
co = cohere.Client('bE6Is3wvtmXyHtgnCQocDIgdH7PcYwdR21ZhnXgN') 

def embeddings(texts,sleep_time=5):
    # add a wait time to simulate a long running process
    time.sleep(sleep_time)
    response = co.embed(
        model='large',
        texts=list(texts), 
        truncate='LEFT').embeddings
    return response

df = pd.read_excel('cohere_docs_embeddings.xlsx')

def load_data(df,):
    df['embeddings'] = embeddings(df['text'])
    # drop rows frm text_df that havve less than 8 words
    df = df[df['text'].str.split().str.len() > 10]
    # Create the search index, pass the size of embedding
    search_index = AnnoyIndex(4096, 'angular')
    # Add all the vectors to the search index, these are stored in the dataframe 'post_members['embeddings']'
    for i, vector in enumerate(df['embeddings']):
        search_index.add_item(i, vector)
    # Build the search index
    search_index.build(10)
    #save the search index
    search_index.save('search_index.ann')
    return df, search_index

df, search_index = load_data(df)

ValueError: Length of values (10) does not match length of index (69)

In [12]:
def search(query, n_results, df, search_index, co):
    # Get the query's embedding
    query_embed = co.embed(texts=[query],
                    model="large",
                    truncate="LEFT").embeddings

    # Get the nearest neighbors and similarity score for the query and the embeddings, append it to the dataframe
    nearest_neighbors = search_index.get_nns_by_vector(query_embed[0], n_results, include_distances=True)
    # filter the dataframe to only include the nearest neighbors using the index
    df = df[df.index.isin(nearest_neighbors[0])]
    df['similarity'] = nearest_neighbors[1]
    df['nearest_neighbors'] = nearest_neighbors[0]
    df = df.sort_values(by='similarity', ascending=False)
    return df


# run function to search for the query
test = search('how do I use model parameters?', 10, df, search_index, co)
test

C:\Users\Elle\AppData\Local\Temp\ipykernel_14488\3330218936.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['similarity'] = nearest_neighbors[1]
C:\Users\Elle\AppData\Local\Temp\ipykernel_14488\3330218936.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nearest_neighbors'] = nearest_neighbors[0]


,text,title,link,embeddings,similarity,nearest_neighbors
58,"['Usage reporting', 'Options', 'get characters...",Usage,/reference/usage,"[2.1582031, -0.003232956, 1.1738281, -0.796875...",1.134067,32
47,['This endpoint makes a prediction about which...,Co.Classifypost,/reference/classify,"[0.4560547, -1.296875, -0.2993164, 0.19396973,...",1.126724,18
43,['This endpoint generates realistic text condi...,Co.Generatepost,/reference/generate,"[2.8222656, -0.94677734, 1.5234375, -0.8032226...",1.114789,42
46,['This endpoint makes a prediction about which...,/classify,/reference/classify,"[0.4560547, -1.296875, -0.2993164, 0.19396973,...",1.114789,43
42,['This endpoint generates realistic text condi...,/generate,/reference/generate,"[2.8222656, -0.94677734, 1.5234375, -0.8032226...",1.107400,19
29,"['This Guide Uses the Classify Endpoint.', 'Yo...",Text Classification (Classify),/docs/text-classification-with-classify,"[0.40716285, -0.7575947, -0.3017723, 0.4197782...",1.099982,46
32,"[""The Responsible Use documentation aims to gu...",Overview,/docs/responsible-use,"[0.7631836, -0.061431885, 0.010261536, 1.24316...",1.099982,47
19,"['In this post, we answer frequently asked que...",Troubleshooting a Custom Model,/docs/training-troubleshooting,"[0.76533026, -0.66085565, 0.5363392, 0.2115352...",1.061606,29
18,"['In this article, we look at training a repre...",Training a Representation Model,/docs/training-a-representation-model,"[-0.2569074, -0.3558816, 1.4379755, -0.1692200...",1.045322,58
17,['Training a generation model consists of a fe...,Training a Generative Model,/docs/training-a-generative-model,"[0.94628906, -1.7255859, 1.1123047, -1.2763672...",0.981783,17
